In [ ]:
%matplotlib inline

import matplotlib
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.path import Path
import matplotlib.patches as patches

from math import sin, cos, pi, hypot
from poisson_disc import poisson_disc
import random
import time
import xy

def circle(cx, cy, r, n):
    result = []
    for i in range(n + 1):
        p = i / float(n)
        a = 2 * pi * p
        x = cx + cos(a) * r
        y = cy + sin(a) * r
        result.append((x, y))
    return result

device = xy.Device( port='/dev/ttyUSB0', baud=115200, up=40, down=10, verbose=False)


In [ ]:
# loading svg and extracting paths
# /home/marcel/projects/composition37/composition37_01.svg
from xml.dom import minidom
doc = minidom.parse('/home/marcel/projects/composition37/composition37_18_option.svg')
path_strings = [path.getAttribute('points') for path
                in doc.getElementsByTagName('polyline')]
paths = []
for p in path_strings:
    point_list = []
    point = p.split(' ')
    #rint(point)
    for _p in point:
        if ',' in _p:
            points = _p.split(',')
            x = float(points[0])
            y = float(points[1])
            #rint(x, y)
            point_list.append((x, y))
            #points = str(points)
            
    paths.append(point_list)
print(len(paths))

fig = plt.figure()
ax = fig.add_subplot(111)


for path in paths:    
    codes = [Path.MOVETO]
    for point in path:
        codes.append(Path.LINETO)
    del codes[-1]

    p = Path(path, codes)
    patch = patches.PathPatch(p, facecolor='none', lw=1)
    ax.add_patch(patch)

plt.axis('equal')
plt.show()

In [ ]:
device.pen_up()

In [ ]:
device.pen_down()

In [ ]:
#device.home()

In [ ]:
device.move(0, 0)

In [ ]:
def bounding_box(paths, scale):
    minx = 1000
    miny = 1000
    maxx = 0
    maxy = 0
    for path in paths:
        for point in path:
            x = point[0] * scale
            y = point[1] * scale
            if x < minx:
                minx = x
            if x > maxx:
                maxx = x
            if y < miny:
                miny = y
            if y > maxy:
                maxy = y
    return [miny, maxy, minx, maxx]

def corners(paths, scale):
    bb = bounding_box(paths, scale)
    return [(bb[0], bb[2]), (bb[0], bb[3]), (bb[1], bb[2]), (bb[1], bb[3])]

def calc_center(paths, scale):
    bb = bounding_box(paths, scale)
    centerx = ((bb[1] - bb[0])/2)+bb[0]
    centery = ((bb[3] - bb[2])/2)+bb[2]
    return [centerx, centery]

bb = bounding_box(paths, 0.15)
cent = calc_center(paths, 0.15)
corn = corners(paths, 0.15)
print('bounding box', bb)
print('center', cent)
print('corners', corn)

In [ ]:
# testing the drawing setup
scale = 0.13
cent = calc_center(paths, scale)
device.pen_up()
device.move(cent[0], cent[1])
def drive_bb(paths, scale):
    c = corners(paths, scale)
    for point in c:
        device.move(point[0], point[1])

drive_bb(paths, scale)
device.move(cent[0], cent[1])

In [ ]:
print('Paths: ', len(paths))
paths.reverse()
index = 0
for path in paths:
    device.pen_up()
    start_x = path[0][1] * scale
    start_y = path[0][0] * scale
    device.move(start_x, start_y)
    print('Starting new path at: ', start_x, start_y)
    device.pen_down()
    for point in path:
        x = point[1] * scale
        y = point[0] * scale
        device.move(x, y)
    device.pen_up()
    index = index + 1
    print('Done index ', index)
    